In [1]:
#Import the pyGSTi module -- you probably want this at the beginning of every notebook
import pygsti
import pygsti.construction as pc

## Creating Gatesets

There are more or less three ways to create GateSet objects in GST:

*  By creating an empty GateSet and populating with calls to `build_gate` and `build_vector`


* By a single call to `build_gateset`, which automates the above approach


* By loading from a text-format gateset file using `pygsti.io.load_gateset`

We'll do each of these in turn.

In [3]:
#1) Create an empty gateset and build gates, prep-vectors, and effect-vectors manually
stateSpace = [2] #density matrix is a 2x2 matrix
spaceLabels = [('Q0',)] #interpret the 2x2 density matrix as a single qubit named 'Q0'

#Initialize an empty GateSet object
gateset1 = pygsti.objects.GateSet()

#Populate the GateSet object with states, effects, gates, and SPAM definitions
gateset1['rho'] = pc.build_vector(stateSpace,spaceLabels,"0")
gateset1['E'] = pc.build_vector(stateSpace,spaceLabels,"1")
gateset1['Gi'] = pc.build_gate(stateSpace,spaceLabels,"I(Q0)")
gateset1['Gx'] = pc.build_gate(stateSpace,spaceLabels,"X(pi/2,Q0)")
gateset1['Gy'] = pc.build_gate(stateSpace,spaceLabels,"Y(pi/2,Q0)")
gateset1['identity'] = pc.build_identity_vec(stateSpace)
gateset1.spamdefs['plus'] = ('rho','E')
gateset1.spamdefs['minus'] = ('rho','remainder')


In [4]:
#2) Create a gateset in a single call to build_gateset
gateset2 = pc.build_gateset( [2], [('Q0',)],['Gi','Gx','Gy'], 
                             [ "I(Q0)","X(pi/2,Q0)", "Y(pi/2,Q0)"],
                             prepLabels = ['rho'], prepExpressions=["0"], 
                             effectLabels = ['E'], effectExpressions=["1"], 
                             spamdefs={'plus': ('rho','E'), 'minus': ('rho','remainder') }) 

In [5]:
#3) Write a text-format gateset file and read it in.
gateset3_txt = \
"""
# Example text file describing a gateset

# State prepared, specified as a state in the Pauli basis (I,X,Y,Z)
rho0
PauliVec
1/sqrt(2) 0 0 1/sqrt(2)

# State measured as yes outcome, also specified as a state in the Pauli basis
E0
PauliVec
1/sqrt(2) 0 0 -1/sqrt(2)

Gi
PauliMx
1 0 0 0
0 1 0 0
0 0 1 0
0 0 0 1

Gx
PauliMx
1 0 0 0
0 1 0 0
0 0 0 1
0 0 -1 0

Gy
PauliMx
1 0 0 0
0 0 0 -1
0 0 1 0
0 1 0 0

IDENTITYVEC sqrt(2) 0 0 0
SPAMLABEL plus = rho0 E0
SPAMLABEL minus = rho0 remainder
"""
open("tutorial_files/Example_Gateset.txt","w").write(gateset3_txt)

gateset3 = pygsti.io.load_gateset("tutorial_files/Example_Gateset.txt")

In [6]:
#All three of the above gatesets are identical.  See for yourself by printing each one:
print "Gateset 1:\n",gateset1
print "Gateset 2:\n",gateset2
print "Gateset 3:\n",gateset3

Gateset 1:
rho =    0.7071        0        0   0.7071


E =    0.7071        0        0  -0.7071


Gi = 
   1.0000        0        0        0
        0   1.0000        0        0
        0        0   1.0000        0
        0        0        0   1.0000


Gx = 
   1.0000        0        0        0
        0   1.0000        0        0
        0        0        0  -1.0000
        0        0   1.0000        0


Gy = 
   1.0000        0        0        0
        0        0        0   1.0000
        0        0   1.0000        0
        0  -1.0000        0        0



Gateset 2:
rho =    0.7071        0        0   0.7071


E =    0.7071        0        0  -0.7071


Gi = 
   1.0000        0        0        0
        0   1.0000        0        0
        0        0   1.0000        0
        0        0        0   1.0000


Gx = 
   1.0000        0        0        0
        0   1.0000        0        0
        0        0        0  -1.0000
        0        0   1.0000        0


Gy = 
   1.0000      

## Basic Operations with Gatesets

`pygsti` `GateSet` objects support a variety of operations, including:

* Depolarizing every gate


* Writing the gate set to a file


* Providing more information about the gate set

In [7]:
#Add 10% depolarization noise to the gates
depol_gateset3 = gateset3.depolarize(gate_noise=0.1)

In [8]:
#Writing a gateset as a text file
pygsti.io.write_gateset(depol_gateset3, "tutorial_files/Example_depolarizedGateset.txt", title="My Gateset")

In [9]:
#Printing more detailed information about a gateset
depol_gateset3.print_info()

rho0 =    0.7071        0        0   0.7071


E0 =    0.7071        0        0  -0.7071


Gi = 
   1.0000        0        0        0
        0   0.9000        0        0
        0        0   0.9000        0
        0        0        0   0.9000


Gx = 
   1.0000        0        0        0
        0   0.9000        0        0
        0        0        0   0.9000
        0        0  -0.9000        0


Gy = 
   1.0000        0        0        0
        0        0        0  -0.9000
        0        0   0.9000        0
        0   0.9000        0        0





Choi Matrices:
Choi(Gi) in pauli basis = 
  --eigenvals =  [0.024999999999999977, 0.024999999999999998, 0.024999999999999998, 0.92500000000000016] 

Choi(Gx) in pauli basis = 
  --eigenvals =  [0.025000000000000005, 0.025000000000000005, 0.025000000000000071, 0.92500000000000049] 

Choi(Gy) in pauli basis = 
  --eigenvals =  [0.025000000000000005, 0.025000000000000005, 0.025000000000000071, 0.92500000000000049] 

Sum of negative Choi e

In [10]:
#If you want to check whether the depolarization procedure actually worked,
#you can run this code.

import numpy as np
def SPAM( G ):
    return np.dot( G['rho0'].T, G['E0'])
dp3 = depol_gateset3.copy()
#dp3.set_evec( dp3.get_rho_vec(0), 0)
dp4 = dp3.depolarize(spam_noise=0.1)
SPAM(dp4)

array([[ 0.095]])

In [11]:
print depol_gateset3


rho0 =    0.7071        0        0   0.7071


E0 =    0.7071        0        0  -0.7071


Gi = 
   1.0000        0        0        0
        0   0.9000        0        0
        0        0   0.9000        0
        0        0        0   0.9000


Gx = 
   1.0000        0        0        0
        0   0.9000        0        0
        0        0        0   0.9000
        0        0  -0.9000        0


Gy = 
   1.0000        0        0        0
        0        0        0  -0.9000
        0        0   0.9000        0
        0   0.9000        0        0



